In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-v0.1"
base_tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# Needed for LLaMA tokenizer
base_tokenizer.pad_token = base_tokenizer.eos_token

from tokenizer import tokenize

ids = tokenize(3.245, base_tokenizer)
print(base_tokenizer.convert_ids_to_tokens(ids))

/home/marc_f/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:664: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


['3', '.', '2', '4', '5']


In [2]:
import numpy as np
ids = tokenize(np.arange(10)**1.5,base_tokenizer)
print(base_tokenizer.convert_ids_to_tokens(ids))

['0', '.', '0', '0', '0', ',', '1', '.', '0', '0', '0', ',', '2', '.', '8', '2', '8', ',', '5', '.', '1', '9', '6', ',', '8', '.', '0', '0', '0', ',', '1', '1', '.', '1', '8', '0', ',', '1', '4', '.', '6', '9', '7', ',', '1', '8', '.', '5', '2', '0', ',', '2', '2', '.', '6', '2', '7', ',', '2', '7', '.', '0', '0', '0']


In [ ]:
import argparse
import bitsandbytes as bnb
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments



def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    print("Number of GPUs:", n_gpus)
    max_memory = f'{9060}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

In [5]:
model_name = "meta-llama/Llama-2-13b-hf" 
#model_name = "bigscience/T0pp"
#model_name = "mistralai/Mistral-7B-v0.1"
bnb_config = create_bnb_config()

model, tokenizer = load_model(model_name, bnb_config)

Number of GPUs: 6


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]